In [ ]:
import fancyimpute
from fancyimpute import KNN

In [ ]:
# Handling outliers in age
def age_nan(age):
    if age < 21:
        return np.nan
    else:
        return age
    
def df_age_nan(df):
    df['age'] = df['age'].apply(lambda x: age_nan(x))
    return df

# Handling outliers in RevolvingUtilizationOfUnsecuredLines
def rev_nan(rev):
    if rev > 2:
        return np.nan
    else:
        return rev
    
def df_rev_nan(df):
    df['RevolvingUtilizationOfUnsecuredLines'] = df['RevolvingUtilizationOfUnsecuredLines'].apply(lambda x: rev_nan(x))
    return df

# Handling outliers in NumberRealEstateLoansOrLines
def real_estate_nan(real_estate):
    if real_estate > 30:
        return np.nan
    else:
        return real_estate

def df_real_estate_nan(df):
    df['NumberRealEstateLoansOrLines'] = df['NumberRealEstateLoansOrLines'].apply(lambda x: real_estate_nan(x))
    return df

# Handling outliers in DebtRatio
def debt_nan(debt):
    if debt > 2:
        return np.nan
    else:
        return debt
    
def df_debt_nan(df):
    df['DebtRatio'] = df['DebtRatio'].apply(lambda x: debt_nan(x))
    return df

# Handling outliers in MonthlyIncome
def income_nan(income):
    if income > 10000:
        return np.nan
    else:
        return income

def df_income_nan(df):
    df['MonthlyIncome'] = df['MonthlyIncome'].apply(lambda x: (income_nan(x)))
    return df

# Handling outliers in NumberOfTime30-59DaysPastDueNotWorse
def late_30(count):
    if count > 13:
        return np.nan
    else:
        return count
    
def df_late_30(df):
    df['NumberOfTime30-59DaysPastDueNotWorse'] = df['NumberOfTime30-59DaysPastDueNotWorse'].apply(lambda x: late_30(x))
    return df

# Handling outliers in NumberOfTime60-89DaysPastDueNotWorse
def late_60(count):
    if count > 11:
        return np.nan
    else:
        return count
    
def df_late_60(df):
    df['NumberOfTime60-89DaysPastDueNotWorse'] = df['NumberOfTime60-89DaysPastDueNotWorse'].apply(lambda x: late_60(x))
    return df

# Handling outliers in NumberOfTimes90DaysLate
def late_90(count):
    if count > 17:
        return np.nan
    else:
        return count
    
def df_late_90(df):
    df['NumberOfTimes90DaysLate'] = df['NumberOfTimes90DaysLate'].apply(lambda x: late_90(x))
    return df

# Handling outliers in NumberOfDependents
def depend(count):
    if count > 6:
        return np.nan
    else:
        return count
    
def df_depend(df):
    df['NumberOfDependents'] = df['NumberOfDependents'].apply(lambda x: depend(x))
    return df

In [ ]:
# Function to replace all outliers with NaN at once
def replace_with_nan(df):
    df = df_age_nan(df)
    df = df_rev_nan(df)
    df = df_real_estate_nan(df)
    df = df_debt_nan(df)
    df = df_income_nan(df)
    df = df_late_30(df)
    df = df_late_60(df)
    df = df_late_90(df)
    df = df_depend(df)
    return df

### Fancyimpute

In [ ]:
# Create boolean mask of nulls
def nan_tf(df):
    for col in df.columns:
        df[col] = np.isnan(df[col])
    return df

# Returns DataFrame, nulls filled using KNN, k=3, clearly does not work
# the way I intended because it fills in 0 for the missing value of age
def df_fancyimpute(df):
    df_mask = nan_tf(df.copy())
    filled = KNN(k=3).fill(df, df_mask)
    filled = pd.DataFrame(filled)
    filled.columns = df.columns
    return filled

In [ ]:
# kernel dies whenever I tries
def fancyimpute2(df):
    X_filled_knn = KNN(k=3).fit_transform(df)
    return X_filled_knn

# Creating Pipeline test for fancyimpute

#### Here is what my next code needs to do:
- Drop the outcome variable and divide the data into 15 groups and label them 1-15.
- Mark things as missing and then create a boolean mask to show where those missing points are.
- Impute them with knn.
- Add the max to all of them so that they are still higher than the other variables.

In [ ]:
testing_fancy = credit.copy()
testing_fancy = testing_fancy.drop(columns='SeriousDlqin2yrs')

testing_fancy = replace_with_nan(testing_fancy)

max_vals = {'NumberOfTime30-59DaysPastDueNotWorse' : 13,
            'NumberOfTime60-89DaysPastDueNotWorse' : 11,
            'NumberOfTimes90DaysLate' : 17,
            'RevolvingUtilizationOfUnsecuredLines' : 2,
            'NumberRealEstateLoansOrLines' : 30,
            'DebtRatio' : 2,
            'MonthlyIncome' : 10000,
            'NumberOfDependents' : 6,
}

testing_fancy_subsets = []
for n in np.arange(1,16):
    testing_fancy_subsets.append(testing_fancy.iloc[(n-1)*10000:n*10000, :])
    
testing_fancy_filled_subsets = []
for subset in testing_fancy_subsets:
    testing_fancy_filled_subsets.append(fancyimpute2(subset))
    
filled_df_subsets = []
for item in testing_fancy_filled_subsets:
    filled_df_subsets.append(pd.DataFrame(item))
    
filled_df = pd.concat(filled_df_subsets)

filled_df.columns = testing_fancy.columns
filled_df.index += 1

filled_df['SeriousDlqin2yrs'] = credit['SeriousDlqin2yrs']

def false_to_zero(element):
    if type(element) != int and type(element) != float:
        element = 0
        return element
    else:
        return element
               
for col in max_mask:
    max_mask[col] = max_mask[col].apply(lambda x: false_to_zero(x))
    
def zero_to_nan(num):
    if num == 0:
        return np.nan
    else:
        return num

for col in max_mask:
    max_mask[col] = max_mask[col].apply(lambda x: zero_to_nan(x))
    


In [ ]:
def fancyimpute(df):
    df_subsets = []
    for n in np.arange(1,16):
        df_subsets.append(df.iloc[(n-1)*10000:n*10000, :])

    df_filled_subsets = []
    for subset in df_subsets:
        df_filled_subsets.append(fancyimpute(subset))
    
    filled_dataframes = []
    for filled_subset in df_filled_subsets:
        filled_dataframes.append(pd.DataFrame(filled_subset))
    
    df = pd.concat(filled_dataframes)
    return df

Two variables had outliers that did not seem to be even on the same scale as the rest of the variable. These I will treat as missing and impute using knn.
- RevolvingUtilizationOfUnsecuredLines
- DebtRatio